# Overview
This notebook prepares ramen-ratings.csv for exploration. The code here has been added to wrangle.py as a script.

In [1]:
# imports
import pandas as pd

df = pd.read_csv('ramen-ratings.csv')
df.head(3)

,Review #,Brand,Variety,Style,Country,Stars,Top Ten
0,2580,New Touch,T's Restaurant Tantanmen,Cup,Japan,3.75,NaN
1,2579,Just Way,Noodles Spicy Hot Sesame Spicy Hot Sesame Guan...,Pack,Taiwan,1,NaN
2,2578,Nissin,Cup Noodles Chicken Vegetable,Cup,USA,2.25,NaN


In [2]:
# drop unrated ramen
df = df.drop(df.loc[df.Stars == 'Unrated'].index)

In [3]:
# cast rating column as float
df['Stars'] = df.Stars.astype('float')

In [4]:
# new column for 5-star ramen
df['five_stars'] = df['Stars'] == 5

In [5]:
# show work
df['five_stars'].value_counts()

False    2191
True      386
Name: five_stars, dtype: int64

In [6]:
# drop review num, stars, and top ten columns
df = df.drop(columns=['Review #','Stars','Top Ten'])

In [7]:
# check nulls
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2577 entries, 0 to 2579
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Brand       2577 non-null   object
 1   Variety     2577 non-null   object
 2   Style       2575 non-null   object
 3   Country     2577 non-null   object
 4   five_stars  2577 non-null   bool  
dtypes: bool(1), object(4)
memory usage: 103.2+ KB


In [8]:
# drop nulls in style
df = df.drop(df[df.Style.isna()].index)

In [9]:
# drop duplicates
df = df.drop_duplicates()

In [10]:
# check work
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2564 entries, 0 to 2579
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Brand       2564 non-null   object
 1   Variety     2564 non-null   object
 2   Style       2564 non-null   object
 3   Country     2564 non-null   object
 4   five_stars  2564 non-null   bool  
dtypes: bool(1), object(4)
memory usage: 102.7+ KB


In [11]:
# rename columns
df = df.rename(columns={'Brand':'brand', 'Variety':'product', 'Style':'package', 'Country':'country'})

In [12]:
# check finalized work
df.head(3)

,brand,product,package,country,five_stars
0,New Touch,T's Restaurant Tantanmen,Cup,Japan,False
1,Just Way,Noodles Spicy Hot Sesame Spicy Hot Sesame Guan...,Pack,Taiwan,False
2,Nissin,Cup Noodles Chicken Vegetable,Cup,USA,False
